In [1]:
# merge_db 데이터 분석

from manager import VectorStoreManager

import os
import logging
import faiss
import torch
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
from typing import Dict, Any
from datetime import datetime, timedelta
from uuid import uuid4
import numpy as np
import pickle, json, re
from dateutil.parser import parse
from utils import DocumentV2, MergedDataV2, SafeUnpickler, create_documents, group_news_data_by_date, join_search_data
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

/mnt/nvme02/User/utopiamath/anaconda3/envs/vaiv310/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# langchain imports 
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# embedding 지정
model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

/tmp/ipykernel_1657114/2329120796.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
2025-07-01 08:21:27.019849: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-01 08:21:27.031007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751358087.043389 16571

In [3]:
# global manager 지정
from pathlib import Path
base_dir = Path("faiss_indexes") # 임시 FAISS DB 경로 수정\
global_manager = VectorStoreManager(embeddings, base_dir)

In [6]:
# global_manager.merge_date_range 이용해 데이터 병합

merged_store = global_manager.merge_date_range("20181231", "20190101")

In [7]:
dir(merged_store)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [9]:
merged_store

In [8]:
# 문서 개수
print(len(merged_store.docstore._dict))  # 저장된 문서 개수

AttributeError: 'NoneType' object has no attribute 'docstore'

In [14]:
# 초기 문서 10개 확인
for doc_id, doc in list(merged_store.docstore._dict.items())[:10]:
    print(f"ID: {doc_id}")
    print(f"Content: {doc.page_content[:1000]}")  # 앞 1000자만 출력
    print("-" * 30)

ID: ac01947a-ba5f-4c57-8104-6f50a2fc2f77
Content: Title: Cost of living
Contents: compensation packages for corporate executives maintaining a western lifestyle. The survey incorporates easy-to-understand comparative cost of living indices between cities. The survey allows city-to-city comparisons, but for the purpose of this report all cities are compared to a base city of New York City, which has an index set at 100. The survey has been carried out for more than 30 years. The most recent survey was published in March 2017. Singapore remains the most expensive city in the world for the fourth year running, in a rare occurrence where the entire top five most expensive cities were unchanged from
------------------------------
ID: 0add434b-95f4-4b71-a77f-0f992765d25e
Content: Title: City
Contents: fifth of the population said to live in shantytowns (favelas, poblaciones callampas, etc.). Batam, Indonesia, Mogadishu, Somalia, Xiamen, China and Niamey, Niger, are considered among the world

In [15]:
# 인덱스 정보 확인
print(merged_store.index.ntotal)  # 벡터 개수

255


In [19]:
# 주어진 쿼리와 유사한 문장 검색
query = "Who is the president of the US?"

results = merged_store.similarity_search(query, k=10)
for c, r in enumerate(results):
    print(c+1, 'th obj')
    print(r.page_content)
    print('*'*50)

1 th obj
Title: President of the United States - Wikipedia
Contents: Head of state and government of the United States

The president of the United States (POTUS)[B] is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.

The power of the presidency has grown[12] since the first president, George Washington, took office in 1789.[6] While presidential power has ebbed and flowed over time, the presidency has played an increasing role in American political life since the beginning of the 20th century, carrying over into the 21st century with some expansions during the presidencies of Franklin D. Roosevelt and George W. Bush.[13][14] In modern times, the president is one of the world's most powerful political figures and the leader of the world's only remaining superpower.[15][16][17] As the leader of the nation with the largest economy by nominal

In [ ]:
# 여러번 넣어서 입력이 복잡해졌나. 그러면 중복 내용을 거르는 부분이 필요함. ()


In [21]:
# retrieval.dpr에서 결과 가져오는 방법 분석

from retrieval.dpr import run_dpr_question, load_model

local_model_name = "Facebook/rag-sequence-nq"
retriever, model, tokenizer = load_model(local_model_name)
search_results = run_dpr_question(query, retriever, model, tokenizer)

Index file found at /mnt/nvme02/User/utopiamath/.cache/wiki_dpr/psgs_w100.tsv.pkl


Loading dataset from disk:   0%|          | 0/157 [00:00<?, ?it/s]

pkl 파일이 성공적으로 로드되었습니다.
MODEL_NAME:: /mnt/nvme01/huggingface/models/Facebook/rag-sequence-nq


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

RagRetriever loaded


Some weights of the model checkpoint at /mnt/nvme01/huggingface/models/Facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokeni